In [ ]:
url = 'https://openapi.naver.com/v1/search/image?query=고양이'
import requests
requests.get(url)   # 1)url을 실행 2) 검색결과 성공여부 리턴 -> 200이 성공인데, 뭔가 에러가 났다. open api에 쓸 수 있는 아이디와 패스워드가 있어야 하는데 없어서 ㅠㅠ

In [ ]:
#네이버 API 인증  정보
client_id = "udzt3jcCA8xTKi3UjGH_" 
client_secret = "HAk0g9n8cs"

In [ ]:
api_id_pw = {
    "X-Naver-Client-Id":client_id,
    "X-Naver-Client-Secret":client_secret
}

In [ ]:
url = 'https://openapi.naver.com/v1/search/image?query=고양이'

result = requests.get(url, headers = api_id_pw)  # 1)url에 접속해서 검색결과 가져올 객체 생성 2)result에 저장
                                                # 나중에 result.json()하면 가져옴
result  # 200이 리턴되면 성공!

In [ ]:
result.json()

In [ ]:
result.json()['items']

In [ ]:
for item in result.json()['items']:
    print('-'*120)
    print(item['link'])
    

In [ ]:
url = 'https://openapi.naver.com/v1/search/image?query=고양이&display=100&start=1'
result = requests.get(url, headers = api_id_pw)
for item in result.json()['items']:
    print("item['link'] =", item['link'])
    print('-'*118)

In [ ]:
url = 'https://openapi.naver.com/v1/search/image?query=고양이&display=100&start=101'
result = requests.get(url, headers = api_id_pw)
for item in result.json()['items']:
    print("item['link'] =", item['link'])
    print('-'*118)

In [ ]:
def get_api_result(keyword, display, start):
    url = f'https://openapi.naver.com/v1/search/image?query={keyword}&display={display}&start={start}'
    print('url=', url)
    result = requests.get(url, headers=api_id_pw)   # 검색할 객체가 생성된다. 객체를 result에 저장!
    return result.json()    # 검색은 여기서! 검색결과가 리턴된다.

In [ ]:
get_api_result('강아지', 100, 901)

In [ ]:
def call_and_print(keyword, total_page=10):
    # 네이버 이미지 검색 결과에서 이미지를 다운 받을 url을 저장 할 리스트
    link_list=[]
    # 1에서 부터 total_page(10페이지)+1 미만 만큼 반복
    for page in range(1,total_page+1):
        # 한페이지당 100개의 이미지 정보를 검색
        display = 100
        # 페이지의 시작 인덱스 한페이지당 100개씩 검색 하기 때문에 1페이지는 1, 2페이지는 101, 3페이지는 201 ...
        start = ((page-1)*display)+1
        # get_api_result(keyword, display,start) : 네이버 이미지 검색 API를 호출하고 결과를 리턴받아서 json_obj 에 대입
        json_obj = get_api_result(keyword, display, start )
        # 검색 결과에서 json_obj['items'] 에 이미지를 다운 받을 URL이 저장되 있으므로 json_obj["item"]을 link_list에 저장
        for item in json_obj['items']:
            link_list.append(item['link'])
    # 이미지를 다운로드할 url이 저장된 linl_list를 리턴        
    return link_list

In [ ]:
keyword = "고양이"
# call_and_print 함수를 호출해서 고양이 이미지를 검색하고 이미지를 다운 받을 URL을 리턴 받음
link1=call_and_print(keyword)
link1

In [ ]:
keyword = "강아지"
# 네이버 API에서 강아지 이미지를 검색하고 다운로드 받을 URL을 리턴 받음
link2=call_and_print(keyword)
link2

In [ ]:
from pymongo import MongoClient
from gridfs import GridFS
from bson.objectid import ObjectId
from gridfs import GridFSBucket

In [ ]:
db = MongoClient().python_test
fs = GridFS(db)
bucket = GridFSBucket(db)

In [ ]:
import urllib.request

In [ ]:
for url in link1:
    try:
        image = urllib.request.urlopen(url).read()

        image_name = url.split('/')[-1]
        if image_name.find('?') != -1:
            image_name = image_name.split('?')[0]
        
        image_type = 'jpg'
        if image_name.find('.') != -1:
            image_type = image_name.split('.')[-1]
        
        content_type = f'image/{image_type}'

        grid_in = bucket.open_upload_stream(
            image_name,
            metadata={'contentType':content_type, 'type':'cat'}
        )
        grid_in.write(image)
        grid_in.close()

        print('image_type:', image_type, 'content_type:', content_type, 'image_name:', image_name)

    except:
        print('Error!!!!!')

In [ ]:
for url in link2:
    try:
        image = urllib.request.urlopen(url).read()

        image_name = url.split('/')[-1]
        if image_name.find('?') != -1:
            image_name = image_name.split('?')[0]
        
        image_type = 'jpg'
        if image_name.find('.') != -1:
            image_type = image_name.split('.')[-1]
        
        content_type = f'image/{image_type}'

        grid_in = bucket.open_upload_stream(
            image_name,
            metadata={'contentType':content_type, 'type':'dog'}
        )
        grid_in.write(image)
        grid_in.close()

        print('image_type:', image_type, 'content_type:', content_type, 'image_name:', image_name)

    except:
        print('Error!!!!!')